In [1]:
import sqlite3
import pandas as pd

In [2]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None

In [3]:
conn = sqlite3.connect('parch-and-posey.db')

In [4]:
cursor = conn.cursor()
cursor.execute('''
select * from sqlite_master where type = "table";
''')
columns = [col[0] for col in cursor.description]
data = cursor.fetchall()
cursor.close()

In [5]:
pd.DataFrame(data, columns=columns)

,type,name,tbl_name,rootpage,sql
0,table,web_events,web_events,2,"CREATE TABLE web_events (\tid integer,\taccoun..."
1,table,sales_reps,sales_reps,7,"CREATE TABLE sales_reps (\tid integer,\tname b..."
2,table,region,region,222,"CREATE TABLE region (\tid integer,\tname bpchar)"
3,table,orders,orders,223,"CREATE TABLE orders (\tid integer,\taccount_id..."
4,table,accounts,accounts,583,"CREATE TABLE accounts (\tid integer,\tname bpc..."


# WIKI

https://en.wikipedia.org/wiki/Date_format_by_country

# DATE_TRUNC

The first function you are introduced to in working with dates is DATE_TRUNC.

DATE_TRUNC allows you to truncate your date to a particular part of your date-time column. Common trunctions are day, month, and year. Here is a great blog post by Mode Analytics on the power of this function.

https://mode.com/blog/date-trunc-sql-timestamp-function-count-on/

# DATE_PART

DATE_PART can be useful for pulling a specific portion of a date, but notice pulling month or day of the week (dow) means that you are no longer keeping the years in order. Rather you are grouping for certain components regardless of which year they belonged in.

# MORE

For additional functions you can use with dates, check out the documentation here https://www.postgresql.org/docs/9.1/functions-datetime.html, but the DATE_TRUNC and DATE_PART functions definitely give you a great start!

You can reference the columns in your select statement in GROUP BY and ORDER BY clauses with numbers that follow the order they appear in the select statement. For example

SELECT standard_qty, COUNT(*)

FROM orders

GROUP BY 1 (this 1 refers to standard_qty since it is the first of the columns included in the select statement)

ORDER BY 1 (this 1 refers to standard_qty since it is the first of the columns included in the select statement)

In [6]:
# DATE_TRUNC ("day", occurred_at) AS day

pd.read_sql_query(sql='''
SELECT occurred_at, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 1;
''', con=conn)

,occurred_at,SUM(standard_qty)
0,2013-12-04 04:22:44,0
1,2013-12-04 04:45:54,490
2,2013-12-04 04:53:25,528
3,2013-12-05 20:29:16,0
4,2013-12-05 20:33:56,492
...,...,...
6903,2017-01-01 21:04:25,0
6904,2017-01-01 22:17:26,497
6905,2017-01-01 22:29:50,38
6906,2017-01-01 23:50:16,291


In [7]:
# DATE_TRUNC ("day", occurred_at) AS day
# strftime('%M-%D-%Y', mydate)
# strftime('%Y-%w', mydate)

pd.read_sql_query(sql='''
SELECT strftime('%Y-%m-%d', occurred_at) AS day, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 1;
''', con=conn)

,day,SUM(standard_qty)
0,2013-12-04,1018
1,2013-12-05,492
2,2013-12-06,1692
3,2013-12-08,3877
4,2013-12-09,972
...,...,...
1055,2016-12-29,2745
1056,2016-12-30,3417
1057,2016-12-31,6449
1058,2017-01-01,5243


In [8]:
pd.read_sql_query(sql='''
SELECT strftime('%Y-%m', occurred_at) AS year_month, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 1;
''', con=conn)

,year_month,SUM(standard_qty)
0,2013-12,26554
1,2014-01,26744
2,2014-02,28459
3,2014-03,27165
4,2014-04,26390
5,2014-05,28772
6,2014-06,28240
7,2014-07,29875
8,2014-08,30142
9,2014-09,28562


In [9]:
pd.read_sql_query(sql='''
SELECT strftime('%Y', occurred_at) AS year, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 1;
''', con=conn)

,year,SUM(standard_qty)
0,2013,26554
1,2014,353967
2,2015,477789
3,2016,1074751
4,2017,5285


In [10]:
pd.read_sql_query(sql='''
SELECT strftime('%Y', occurred_at) AS year, SUM(standard_qty)
FROM orders
GROUP BY year
ORDER BY year;
''', con=conn)

,year,SUM(standard_qty)
0,2013,26554
1,2014,353967
2,2015,477789
3,2016,1074751
4,2017,5285


In [11]:
# DATE_PART ("month", occurred_at)

pd.read_sql_query(sql='''
SELECT strftime('%m', occurred_at) AS month, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 1;
''', con=conn)

,month,SUM(standard_qty)
0,01,122046
1,02,116502
2,03,127941
3,04,131582
4,05,135767
5,06,166539
6,07,158189
7,08,164702
8,09,175252
9,10,207647


In [12]:
# DATE_PART ("month", occurred_at)

pd.read_sql_query(sql='''
SELECT strftime('%H', occurred_at) AS hour, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 1;
''', con=conn)

,hour,SUM(standard_qty)
0,00,88332
1,01,52294
2,02,81341
3,03,80611
4,04,94644
5,05,76098
6,06,77563
7,07,77959
8,08,81711
9,09,95524


In [13]:
# DATE_PART ("month", occurred_at)

pd.read_sql_query(sql='''
SELECT strftime('%M', occurred_at) AS minute, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 1;
''', con=conn)

,minute,SUM(standard_qty)
0,00,32220
1,01,30993
2,02,31963
3,03,23135
4,04,40659
5,05,22719
6,06,27657
7,07,29768
8,08,36085
9,09,32139


In [14]:
pd.read_sql_query(sql='''
SELECT strftime('%S', occurred_at) AS second, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 1;
''', con=conn)

,second,SUM(standard_qty)
0,00,38816
1,01,35181
2,02,20738
3,03,30064
4,04,35700
5,05,30848
6,06,27379
7,07,29017
8,08,23764
9,09,31216


In [18]:
# DATE_PART('dow', occurred_at)

pd.read_sql_query(sql='''
SELECT strftime('%w', occurred_at) AS day_of_week, SUM(standard_qty)
FROM orders
GROUP BY 1
ORDER BY 2 DESC;
''', con=conn)

,day_of_week,SUM(standard_qty)
0,5,299578
1,3,286285
2,0,279629
3,6,276572
4,2,271922
5,4,267389
6,1,256971
